In [1]:
from IPython.display import clear_output
import json
import random

# LOAD FILES
with open("Moves.json", "r") as f:
    MOVESET = json.load(f)

with open("Types.json", "r") as f:
    TYPES = json.load(f)

In [2]:
# MOVESET
moveset = {
    1: MOVESET["Heal"],
    2: MOVESET["Attack"],
    3: None,
    4: None
}

In [3]:
# GAME START AND INFO

def state_start(data):
    is_first = random.choice([True, False])
    p_type = TYPES[str(random.choice([0, 2]))]
    e_type = TYPES[str(random.choice([1, 1]))]

    data.update({
        "is_player_first": is_first,
        "last_actor": None,


        "player_hp": p_type["base_hp"],
        "player_type": p_type,
        "player_move": None,

        "enemy_hp": e_type["base_hp"],
        "enemy_type": e_type,
    })

    print("=== BATTLE STARTING ===")
    print(("PLAYER" if is_first else "ENEMY"), "goes first!\n")
    return "DISPLAY_INFO"

def state_display_info(data):
    print("--PLAYER INFO--")
    print("Health:", data["player_hp"])
    print("Type:", data["player_type"]["name"])
    print("\n--ENEMY INFO--")
    print("Health:", data["enemy_hp"])
    print("Type:", data["enemy_type"]["name"])
    print()

    return "INPUT_PLAYER"

In [4]:
# GET PLAYER MOVE SELECTION

def state_input_player(data):
    move_index = int(input("Select a move (1-4): "))
    move = moveset.get(move_index)
    if move is None:
        print("Invalid move! Turn wasted.")
        data['last_actor'] = 'player'
    
    data['player_move'] = move
    
    return "PLAYER_TURN" if data['is_player_first'] else "ENEMY_TURN"
    #print("YOU used:", move["type"])

In [5]:
def get_type_damage_modifier(data, actor):
    EFFECTIVE = 1.2
    INEFFECTIVE = 0.8

    # actor determines who is attacking
    if actor == "player":
        attacker = data["player_type"]
        defender = data["enemy_type"]
    else:  # actor == "enemy"
        attacker = data["enemy_type"]
        defender = data["player_type"]

    # exact same type
    if attacker == defender:
        return 1.0

    # attacker is strong against defender
    if defender == TYPES[attacker["effective"]]:
        return EFFECTIVE

    # attacker is weak against defender
    if defender == TYPES[attacker["ineffective"]]:
        return INEFFECTIVE

    return 1.0


In [6]:
# PLAYER TURN

def state_player_turn(data):
    print("=== PLAYER TURN ===")

    if data['player_move'] == None:
        return "CHECK_WIN" 
    
    ptype = data['player_type']
    if data["player_move"]["type"] == "HEAL":
        data["player_hp"] += ptype['heal_base_hp']
    elif data["player_move"]["type"] == "ATTACK":
        data["enemy_hp"] -= round(ptype['attack_base_hp'] * get_type_damage_modifier(data, 'player'))

    data['last_actor'] = 'player'
    return "CHECK_WIN"

In [7]:
def enemy_attack(data, etype):
    data["player_hp"] -= round(etype["attack_base_hp"]  * get_type_damage_modifier(data, 'enemy'))
    print("ENEMY used: ATTACK")
    data["last_actor"] = "enemy"
    


In [8]:
# ENEMY TURN
def state_enemy_turn(data):
    print("=== ENEMY TURN ===")
    ehp = data["enemy_hp"]
    etype = data["enemy_type"]

    # ATTACK if: base attack dmg > player's hp
    if etype['attack_base_hp'] > data['player_hp']:
        enemy_attack(data, etype)
        return "CHECK_WIN"


    # HEAL if low (probability-based)
    if ehp < (etype["base_hp"] * etype["heal_threshold"]):
        if random.random() < etype["heal_probability"]:
            data["enemy_hp"] += etype["heal_base_hp"]
            print("ENEMY used: HEAL")

            data["last_actor"] = "enemy"
            return "CHECK_WIN"

    # default: ATTACK 
    enemy_attack(data, etype)
    return "CHECK_WIN"

In [9]:
# WIN DETECTION

def state_check_win(data):
    hp = data["player_hp"]
    if hp < 0: hp = 0

    ehp = data["enemy_hp"]
    if ehp < 0: ehp = 0

    print(f"Player HP: {hp}")
    print(f"Enemy  HP: {ehp}\n")

    if hp <= 0:
        print("You lose.")
        return "END"
    if ehp <= 0:
        print("You win!")
        return "END"

    # Continue cycle
    last_actor = data['last_actor']
    starting_actor = 'player' if data['is_player_first'] else 'enemy'
    non_starting_actor = 'ENEMY' if starting_actor == 'player' else 'PLAYER'

    # if turn is over: go back to move selection
    if last_actor != starting_actor:
        return "INPUT_PLAYER"
    # else: go to 2nd actor
    else:
        return f"{non_starting_actor}_TURN"


In [10]:
# END STATE

def state_end(data):
    print("=== BATTLE FINISHED ===")
    return None

In [11]:
# GAME BEGIN

STATES = {
    "START": state_start,
    "DISPLAY_INFO": state_display_info,
    "INPUT_PLAYER": state_input_player,
    "PLAYER_TURN": state_player_turn,
    "ENEMY_TURN": state_enemy_turn,
    "CHECK_WIN": state_check_win,
    "END": state_end,
}


def run_fsm():
    data = {}
    state = "START"

    while state is not None:
        state = STATES[state](data)
        

In [12]:
# Start game
run_fsm()

=== BATTLE STARTING ===
ENEMY goes first!

--PLAYER INFO--
Health: 70
Type: Fire

--ENEMY INFO--
Health: 110
Type: Water

=== ENEMY TURN ===
ENEMY used: ATTACK
Player HP: 58
Enemy  HP: 110

=== PLAYER TURN ===
Player HP: 58
Enemy  HP: 86

=== ENEMY TURN ===
ENEMY used: ATTACK
Player HP: 46
Enemy  HP: 86

=== PLAYER TURN ===
Player HP: 46
Enemy  HP: 62

=== ENEMY TURN ===
ENEMY used: HEAL
Player HP: 46
Enemy  HP: 102

=== PLAYER TURN ===
Player HP: 46
Enemy  HP: 78

=== ENEMY TURN ===
ENEMY used: ATTACK
Player HP: 34
Enemy  HP: 78

=== PLAYER TURN ===
Player HP: 34
Enemy  HP: 54

=== ENEMY TURN ===
ENEMY used: ATTACK
Player HP: 22
Enemy  HP: 54

=== PLAYER TURN ===
Player HP: 22
Enemy  HP: 30

=== ENEMY TURN ===
ENEMY used: ATTACK
Player HP: 10
Enemy  HP: 30

=== PLAYER TURN ===
Player HP: 10
Enemy  HP: 6

=== ENEMY TURN ===
ENEMY used: HEAL
Player HP: 10
Enemy  HP: 46

=== PLAYER TURN ===
Player HP: 20
Enemy  HP: 46



ValueError: invalid literal for int() with base 10: ''